# Day 3: Gear Ratios

[*Advent of Code 2023 day 3*](https://adventofcode.com/2023/day/3) and [*solution megathread*](https://redd.it/189m3qw)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2023/03/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2023%2F03%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')


# %load_ext nb_mypy
# %nb_mypy On

In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

# %load_ext pycodestyle_magic
# %pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


In [3]:
from IPython.display import HTML

HTML(downloaded['part1'])

In [4]:
example_input = '''467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..'''

In [5]:
schematic = example_input.splitlines()
# schematic = downloaded['input'].splitlines()

In [6]:
from typing import List, Iterator, Tuple

def neighbor_coords(schematic: List[str], row: int, column: int) -> Iterator[Tuple[int, int]]:
    size = len(schematic)
    assert(size == len(schematic[0]))
    return (
        (r, c)
        for r, c
        in [
            (row - 1, column - 1), (row - 1, column), (row - 1, column + 1),
            (row, column - 1), (row, column + 1),
            (row + 1, column - 1), (row + 1, column), (row + 1, column + 1)
            ]
        if 0 <= r < size and 0 <= c < size)

In [7]:
def neighbors(schematic: List[str], row: int, column: int) -> Iterator[str]:
    return (
        schematic[r][c]
        for r, c
        in neighbor_coords(schematic, row, column)
        )

print(list(neighbors(schematic, 0, 0)))
print(list(neighbors(schematic, 5, 5)))

['6', '.', '.']
['.', '.', '.', '.', '.', '2', '.', '.']


In [8]:
def find_parts(schematic: List[str]) -> Iterator[Tuple[int, int]]:
    for row, line in enumerate(schematic):
        for column, current in enumerate(line):
            if (
                    not current.isdigit()
                    and current != '.'
                    ):
                yield (row, column)

list(find_parts(schematic))

[(1, 3), (3, 6), (4, 3), (5, 5), (8, 3), (8, 5)]

In [9]:
def find_number(schematic: List[str], row: int, column: int) -> Tuple[int, int, int]:
    size = len(schematic)
    assert(size == len(schematic[0]))
    start_col = column
    stop_col = column
    while start_col > 0 and schematic[row][start_col - 1].isdigit():
        start_col -= 1
    while stop_col + 1 < size and schematic[row][stop_col + 1].isdigit():
        stop_col += 1
    return row, start_col, int(schematic[row][start_col:stop_col+1])

print(find_number(schematic, 0, 0))
print(find_number(schematic, 0, 7))

(0, 0, 467)
(0, 5, 114)


In [10]:
def find_part_numbers(schematic: List[str], row: int, column: int) -> Iterator[Tuple[int, int, int]]:
    return set(
        find_number(schematic, r, c)
        for r, c
        in neighbor_coords(schematic, row, column)
        if schematic[r][c].isdigit()
        )

list(find_part_numbers(schematic, 1, 3))

[(0, 0, 467), (2, 2, 35)]

In [11]:
from functools import partial
from itertools import starmap, chain

def find_parts_numbers(schematic: List[str]) -> Iterator[int]:
    return (number
            for _, _, number
            # Form a set of all numbers with coordinates to only get unique ones
            in set(
                chain(
                    # starmap unpacks row and column from the find_parts iterator,
                    # feeding them to the partial find_part_numbers
                    *starmap(
                        partial(find_part_numbers, schematic),
                        find_parts(schematic)
                        )
                    )
                )
            )

sum(find_parts_numbers(schematic))

4361

In [12]:
HTML(downloaded['part2'])

In [13]:
def find_gears_ratios(schematic: List[str]) -> Iterator[int]:
    for row, column in find_parts(schematic):
        if schematic[row][column] != '*':
            continue
        part_numbers = tuple(find_part_numbers(schematic, row, column))
        if len(part_numbers) != 2:
            continue
        yield part_numbers[0][2] * part_numbers[1][2]

sum(find_gears_ratios(schematic))

467835